<a href="https://colab.research.google.com/github/woosikyang/DB-Server/blob/master/Flask_pytorch_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
from flask import Flask

In [2]:
app = Flask(__name__)

@app.route('/')
def hello() : 
  return 'hello world!'

In [3]:
! FLASK_ENV=development FLASK_APP=app.py flask run


 * Debugger is active!
 * Debugger PIN: 104-700-018
Exception KeyboardInterrupt in <bound method Popen.__del__ of <subprocess.Popen object at 0x7fe5f9600550>> ignored


In [ ]:
@app.route('/predict', methods=['POST'])

from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict() : 
  return jsonify({'class_id' : 'IMAGE_NET_XXX', 'class_name' : 'Cat'})

  

In [ ]:
#inference

import io
import torchvision.transforms as transforms
from PIL import Image

def transform_image(image_bytes) :
  my = transforms.Compose([transforms.Resize(255),
                           transforms.CenterCrop(224),
                           transforms.ToTensor(),
                           transforms.Normalize(
                               [0.485, 0.456, 0.406],
                               [0.229, 0.224, 0.225])])
  image = Image.open(io.BytesIO(image_bytes))
  return my_transforms(image).unsqueeze(0)


In [4]:
from torchvision import models

model = models.densenet121(pretrained=True)

model.eval()


def get_prediction(image_bytes) : 
  tensor = transform_image(image_bytes = image_bytes)
  outputs = model.forward(tensor)
  _, y_hat = outputs.max(1)
  return y_hat

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/checkpoints/densenet121-a639ec97.pth


In [ ]:
from flask import request

@app.route('/predict', methods=['POST'])

def predict() :
  if request.method == 'POST' :
    file = request.files['file']
    img_bytes = file.read()
    class_id, class_name = get_prediction(image_bytes=img_bytes)

    return jsonify({'class_id' : class_id, 'class_name' : class_name})




if __name__ == '__main__' :
  app.run()

In [ ]:
import json

from torchvision import models
import torchvision.transforms as transforms
from PIL import Image
from flask import Flask, jsonify, request


app = Flask(__name__)
imagenet_class_index = json.load(open('<PATH/TO/.json/FILE>/imagenet_class_index.json'))
model = models.densenet121(pretrained=True)
model.eval()


def transform_image(image_bytes):
    my_transforms = transforms.Compose([transforms.Resize(255),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize(
                                            [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])])
    image = Image.open(io.BytesIO(image_bytes))
    return my_transforms(image).unsqueeze(0)


def get_prediction(image_bytes):
    tensor = transform_image(image_bytes=image_bytes)
    outputs = model.forward(tensor)
    _, y_hat = outputs.max(1)
    predicted_idx = str(y_hat.item())
    return imagenet_class_index[predicted_idx]


@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        file = request.files['file']
        img_bytes = file.read()
        class_id, class_name = get_prediction(image_bytes=img_bytes)
        return jsonify({'class_id': class_id, 'class_name': class_name})


if __name__ == '__main__':
    app.run()